# Machine Learning Module (exercise solutions)

**Lecturer:** Ashish Mahabal<br>
**Jupyter Notebook Authors:** Ashish Mahabal

This is a Jupyter notebook lesson taken from the GROWTH Summer School 2019.  For other lessons and their accompanying lectures, please see: http://growth.caltech.edu/growth-school-2019.html

## Objective
Classify different classes using (a) decision trees and (b) random forest 

## Key steps
- Pick variable types
- Select training sample
- Select method
- Look at confusion matrix and details 

## Required dependencies

See GROWTH school webpage for detailed instructions on how to install these modules and packages.  Nominally, you should be able to install the python modules with `pip install <module>`.  The external astromatic packages are easiest installed using package managers (e.g., `rpm`, `apt-get`).

### Python modules
* python 3
* astropy
* numpy
* astroquery
* pandas
* matplotlib
* pydotplus
* IPython.display
* sklearn

### External packages
None

### Partial Credits
Pavlos Protopapas (LSSDS notebook)

### Here you will use the light curves file to derive features
### And then use the resulting file to run decision trees and random forest on that for classification

#### import the required modules (exercise)
#### The exercise contained only a couple of imports.
#### Reproduced below are all the libraries you will need
### Remember to install graphviz and pydot if you had issues before

In [1]:
# For inline plots
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import io
import pydotplus
from IPython.display import Image

# Various scikit-learn modules
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

#### read the lightcurves file
#### This is exactly like before except for one main difference
### We are *not* restricting ourselves to just 100k rows!

In [2]:
datadir = 'data'
lightcurves = datadir + '/CRTS_6varclasses.csv.gz'

In [3]:
lcs = pd.read_csv(lightcurves,
                 compression='gzip',
                 header=1,
                 sep=',',
                 skipinitialspace=True)
                 #nrows=100000)
                 #skiprows=[4,5])
                 #,nrows=100000)

lcs.columns = ['ID', 'MJD', 'Mag', 'magerr', 'RA', 'Dec']
lcs.head()

,ID,MJD,Mag,magerr,RA,Dec
0,1109065026725,53705.501925,16.943797,0.082004,182.25871,9.76580
1,1109065026725,53731.483314,16.645102,0.075203,182.25867,9.76585
2,1109065026725,53731.491406,16.693791,0.076497,182.25870,9.76574
3,1109065026725,53731.499465,16.793651,0.078755,182.25869,9.76576
4,1109065026725,53731.507529,16.767817,0.077436,182.25878,9.76581


In [4]:
len(lcs)

4256337

#### We need classes, so load the catalog file too
#### This too is like before
#### We will call our dataframe 'cat'

In [5]:
catalog = datadir + '/CatalinaVars.tbl.gz'

In [6]:
cat = pd.read_csv(catalog,
                 compression='gzip',
                 header=5,
                 sep=' ',
                 skipinitialspace=True,
                 )

columns = cat.columns[1:]
cat = cat[cat.columns[:-1]]
cat.columns = columns

cat.head()

,Catalina_Surveys_ID,Numerical_ID,RA_J2000,Dec,V_mag,Period_days,Amplitude,Number_Obs,Var_Type
0,CSS_J000020.4+103118,1109001041232,00:00:20.41,+10:31:18.9,14.62,1.491758,2.39,223,2
1,CSS_J000031.5-084652,1009001044997,00:00:31.50,-08:46:52.3,14.14,0.404185,0.12,163,1
2,CSS_J000036.9+412805,1140001063366,00:00:36.94,+41:28:05.7,17.39,0.274627,0.73,158,1
3,CSS_J000037.5+390308,1138001069849,00:00:37.55,+39:03:08.1,17.74,0.30691,0.23,219,1
4,CSS_J000103.3+105724,1109001050739,00:01:03.37,+10:57:24.4,15.25,1.5837582,0.11,223,8


#### Some of the following steps are not really needed as we have developed some functions combining multiple things already.
#### But yu could just use these to restrict the set in some ways
### For example, those having at least 100 observations

In [7]:
RRd = cat[ cat['Var_Type'].isin([6]) & (cat['Number_Obs']>100) ]

In [8]:
RRd.head()

,Catalina_Surveys_ID,Numerical_ID,RA_J2000,Dec,V_mag,Period_days,Amplitude,Number_Obs,Var_Type
115,CSS_J001420.8+031214,1104002007409,00:14:20.84,+03:12:14.0,17.45,0.3871100,0.56,174,6
198,CSS_J001724.9+200542,1121002007726,00:17:24.90,+20:05:42.2,16.64,0.3571291,0.39,224,6
214,CSS_J001812.9+210201,1121002027610,00:18:12.97,+21:02:01.5,14.54,0.41616,0.34,224,6
531,CSS_J003001.7+094947,1109003028079,00:30:01.71,+09:49:47.6,16.91,0.3729404,0.36,212,6
640,CSS_J003359.4+022609,1101004049971,00:33:59.48,+02:26:09.0,15.87,0.3601025,0.27,195,6


### Get numerical ids of objects belonging to the RRd class - call them RRds

In [9]:
RRds = RRd['Numerical_ID']

In [10]:
RRds.head()

115    1104002007409
198    1121002007726
214    1121002027610
531    1109003028079
640    1101004049971
Name: Numerical_ID, dtype: int64

### Let us extract some features from the mags (lets ignore the mag errors for now)

#### For a given id you could do it as follows. isin() accepts a list so you could use the entire RRds there
#### you will lose the id information if you do that in a single step, so you could break it up

In [11]:
lcs[lcs['ID'].isin(['1109065026725'])]['Mag']

0      16.943797
1      16.645102
2      16.693791
3      16.793651
4      16.767817
5      16.885437
6      16.845561
7      16.888531
8      16.941978
9      16.822148
10     16.847925
11     16.878077
12     16.879755
13     16.889322
14     16.943497
15     16.883803
16     16.900262
17     16.742775
18     16.877542
19     16.952343
20     16.906344
21     16.713005
22     16.842459
23     16.895991
24     16.738557
25     16.860171
26     16.881559
27     16.753773
28     16.930406
29     16.613862
         ...    
369    16.476747
370    16.497292
371    16.866625
372    16.930688
373    16.877139
374    16.933141
375    16.885875
376    16.729198
377    16.851957
378    16.861757
379    16.702795
380    16.797014
381    16.781394
382    16.806467
383    16.517148
384    16.556145
385    16.533153
386    16.561263
387    16.462612
388    16.483990
389    16.489717
390    16.485392
391    16.487658
392    16.490796
393    16.474579
394    16.461535
395    16.891826
396    16.8245

In [12]:
lcs[lcs['ID'].isin(RRds)]['Mag']

1212       14.998325
1213       14.984769
1214       15.010683
1215       14.984963
1216       14.817003
1217       14.818442
1218       14.824113
1219       14.816897
1220       14.828202
1221       14.826791
1222       14.808851
1223       14.838217
1224       14.800337
1225       14.813361
1226       14.785231
1227       14.799678
1228       14.925248
1229       14.925225
1230       14.914077
1231       14.905855
1232       14.830322
1233       14.843997
1234       14.837703
1235       14.818362
1236       14.961875
1237       14.988142
1238       14.979043
1239       14.979344
1240       14.869105
1241       14.854159
             ...    
4249204    14.465706
4249205    14.470303
4249206    14.366543
4249207    14.374944
4249208    14.363363
4249209    14.372948
4249210    14.223580
4249211    14.219513
4249212    14.240163
4249213    14.247493
4249214    14.249372
4249215    14.264992
4249216    14.301054
4249217    14.325370
4249218    14.570421
4249219    14.575062
4249220    14

#### Let us assign mags for '1109065026725' to mags (a dictionary)

In [13]:
mags = {}
mags['1109065026725'] = lcs[lcs['ID'].isin(['1109065026725'])]['Mag']

#### Let us get the mean of mags for this one particular object: '1109065026725'

In [14]:
np.mean(mags['1109065026725'].values)

16.717012834586466

#### Assign it to another dictionary with the same key

In [15]:
means = {}
means['1109065026725'] = np.mean(mags['1109065026725'].values)

## Exercise!

### Get mean, median, skew, kurtosis for all ids in our light curves set

## To clarify, don't just execute the cell below, but add these columns to your dataset just like we added the 'target' column in the other notebook.
## Use the definitions given below for skew, kurtosis, median, and the one given above for mean

In [16]:
from scipy.stats import skew, kurtosis
skew(mags['1109065026725'])
kurtosis(mags['1109065026725'])
np.median(mags['1109065026725'])

16.729198

### Here we define the dictionaries, and add the corresponding values for ALL lightcurves (almost 50K)
## NOTE: This can take several minutes - be patient
## If needed add a variable that prints a bit after every 1000 lightcurves

In [17]:
meanvals = {}
skewvals = {}
kurtosisvals = {}
medianvals ={}
for id in cat['Numerical_ID']:
    mags = lcs[lcs['ID'].isin([id])]['Mag']
    skewvals[id] = skew(mags)
    medianvals[id] = np.median(mags)
    kurtosisvals[id] = kurtosis(mags)
    meanvals[id] = np.mean(mags)

/Users/igor/Software/ZTF/miniconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/igor/Software/ZTF/miniconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


### Add the column defined by the dict into our main dataframe

In [18]:
cat['median'] = cat['Numerical_ID'].map(medianvals)

### Add the three other columns similarly

In [19]:
cat['mean'] = cat['Numerical_ID'].map(meanvals)
cat['skew'] = cat['Numerical_ID'].map(skewvals)
cat['kurtosis'] = cat['Numerical_ID'].map(kurtosisvals)

#### Check the data

In [20]:
len(cat)

47055

In [21]:
cat.head()

,Catalina_Surveys_ID,Numerical_ID,RA_J2000,Dec,V_mag,Period_days,Amplitude,Number_Obs,Var_Type,median,mean,skew,kurtosis
0,CSS_J000020.4+103118,1109001041232,00:00:20.41,+10:31:18.9,14.62,1.491758,2.39,223,2,14.516051,14.652289,3.947630,14.644276
1,CSS_J000031.5-084652,1009001044997,00:00:31.50,-08:46:52.3,14.14,0.404185,0.12,163,1,NaN,NaN,NaN,NaN
2,CSS_J000036.9+412805,1140001063366,00:00:36.94,+41:28:05.7,17.39,0.274627,0.73,158,1,NaN,NaN,NaN,NaN
3,CSS_J000037.5+390308,1138001069849,00:00:37.55,+39:03:08.1,17.74,0.30691,0.23,219,1,NaN,NaN,NaN,NaN
4,CSS_J000103.3+105724,1109001050739,00:01:03.37,+10:57:24.4,15.25,1.5837582,0.11,223,8,15.252232,15.255873,8.203262,98.291603


### Uh-oh! we have been 'NaN'ed

### Actually what was delibretaly done - to reduce the number of light curves to be managable - was to take out the biggest class, class 1 (otherwise your features could have taken longer - our code is not the most efficient one as we get one light curve at a time)

### Now create a file with the following columns
### ID, mean, median, skew, Kurtosis, Class

In [22]:
myfile = cat[['Numerical_ID','Var_Type','mean','median','skew','kurtosis']]

In [23]:
myfile.head()

,Numerical_ID,Var_Type,mean,median,skew,kurtosis
0,1109001041232,2,14.652289,14.516051,3.947630,14.644276
1,1009001044997,1,NaN,NaN,NaN,NaN
2,1140001063366,1,NaN,NaN,NaN,NaN
3,1138001069849,1,NaN,NaN,NaN,NaN
4,1109001050739,8,15.255873,15.252232,8.203262,98.291603


### Let us get just classes 2 and 4. Though, we do not need that
### as we have already defined a function that lets us
### take any pair of classes at will.
### We pluck 2 and 4 to make a point 

In [24]:
vars2 = myfile[ myfile['Var_Type'].isin([2,4])  ]
vars2.head()

,Numerical_ID,Var_Type,mean,median,skew,kurtosis
0,1109001041232,2,14.652289,14.516051,3.947630,14.644276
23,1118001060639,2,17.846677,17.798409,1.725058,3.901943
28,1112001023767,2,16.714586,16.666647,2.464050,6.551792
30,1012001026394,4,18.545355,18.566653,0.024304,-0.415180
32,1143001058200,4,NaN,NaN,NaN,NaN


## Ah! There are NaNs here too (exercise: why?)

In [25]:
len(vars2)

7114

### We drop them

In [26]:
vars2nonnan = vars2.dropna()

In [27]:
len(vars2nonnan)

6264

In [28]:
vars2nonnan.head()

,Numerical_ID,Var_Type,mean,median,skew,kurtosis
0,1109001041232,2,14.652289,14.516051,3.947630,14.644276
23,1118001060639,2,17.846677,17.798409,1.725058,3.901943
28,1112001023767,2,16.714586,16.666647,2.464050,6.551792
30,1012001026394,4,18.545355,18.566653,0.024304,-0.415180
43,1018001037204,4,19.278016,19.249004,0.193230,0.002537


## Now run decision tree and random forest using these variables by picking a couple of classes. We bring in our definitions again

In [29]:
def display_dt(dt):
    dummy_io = io.StringIO() 
    tree.export_graphviz(dt, out_file = dummy_io, proportion=True) 
    print(dummy_io.getvalue())

In [30]:
# This function creates images of tree models using pydotplus
# https://github.com/JWarmenhoven/ISLR-python
def print_tree(estimator, features, class_names=None, filled=True):
    tree = estimator
    names = features
    color = filled
    classn = class_names
    
    dot_data = io.StringIO()
    export_graphviz(estimator, out_file=dot_data, feature_names=features, proportion=True, class_names=classn, filled=filled)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
    return(graph)

In [31]:
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #
# Important parameters
# indf - Input dataframe
# featurenames - vector of names of predictors
# targetname - name of column you want to predict (e.g. 0 or 1, 'M' or 'F', 
#              'yes' or 'no')
# target1val - particular value you want to have as a 1 in the target
# mask - boolean vector indicating test set (~mask is training set)
# reuse_split - dictionary that contains traning and testing dataframes 
#              (we'll use this to test different classifiers on the same 
#              test-train splits)
# score_func - we've used the accuracy as a way of scoring algorithms but 
#              this can be more general later on
# n_folds - Number of folds for cross validation ()
# n_jobs - used for parallelization
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #

def do_classify(clf, parameters, indf, featurenames, targetname, target1val, mask=None, reuse_split=None, score_func=None, n_folds=5, n_jobs=1):
    subdf=indf[featurenames]
    X=subdf.values
    y=(indf[targetname].values==target1val)*1
    if mask.any() !=None:
        print("using mask")
        Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]
    if reuse_split !=None:
        print("using reuse split")
        Xtrain, Xtest, ytrain, ytest = reuse_split['Xtrain'], reuse_split['Xtest'], reuse_split['ytrain'], reuse_split['ytest']
    if parameters:
        clf = cv_optimize(clf, parameters, Xtrain, ytrain, n_jobs=n_jobs, n_folds=n_folds, score_func=score_func)
    clf=clf.fit(Xtrain, ytrain)
    training_accuracy = clf.score(Xtrain, ytrain)
    test_accuracy = clf.score(Xtest, ytest)
    print("############# based on standard predict ################")
    print("Accuracy on training data: %0.2f" % (training_accuracy))
    print("Accuracy on test data:     %0.2f" % (test_accuracy))
    print(confusion_matrix(ytest, clf.predict(Xtest)))
    print("########################################################")
    return(clf, Xtrain, ytrain, Xtest, ytest)

### Including the one where we combined multiple steps

In [32]:
def dtclassify(allclasses,class1,class2,var1,var2):
    vars2 = allclasses[ allclasses['Var_Type'].isin([class1,class2])  ]
    Y = vars2['Var_Type'].values
    Y = np.array([1 if y==class1 else 0 for y in Y])
    X = vars2.drop('Var_Type',1).values
    vars2['target'] = (vars2['Var_Type'].values==class1)*1
    
    # Create test/train mask
    itrain, itest = train_test_split(range(vars2.shape[0]), train_size=0.6)
    mask=np.ones(vars2.shape[0], dtype='int')
    mask[itrain]=1
    mask[itest]=0
    mask = (mask==1)
    
    print("% Class ",class1," objects in Training:", np.mean(vars2.target[mask]), np.std((vars2.target[mask])))
    print("% Class ",class2," objects in Testing:", np.mean(vars2.target[~mask]), np.std((vars2.target[~mask])))
    
    clfTree1 = tree.DecisionTreeClassifier(max_depth=3, criterion='gini')

    subdf=vars2[[var1, var2]]
    X=subdf.values
    y=(vars2['target'].values==1)*1

    # TRAINING AND TESTING
    Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]

    # FIT THE TREE 
    clf=clfTree1.fit(Xtrain, ytrain)

    training_accuracy = clf.score(Xtrain, ytrain)
    test_accuracy = clf.score(Xtest, ytest)
    print("############# based on standard predict ################")
    print("Accuracy on training data: %0.2f" % (training_accuracy))
    print("Accuracy on test data:     %0.2f" % (test_accuracy))
    print(confusion_matrix(ytest, clf.predict(Xtest)))
    print("########################################################")
    
    display_dt(clf)
    return [clf,var1,var2]
    
#    graph3 = print_tree(clf, features=[var1, var2], class_names=['No', 'Yes'])
#    Image(graph3.create_png())
    


In [33]:
# A generic function to do CV

def cv_optimize(clf, parameters, X, y, n_jobs=1, n_folds=5, score_func=None):
    if score_func:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, scoring=score_func)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, n_jobs=n_jobs, cv=n_folds)
    gs.fit(X, y)

    best = gs.best_estimator_
    return best

### Run the dt classifier for classes 2 and 4 using two of the features we defined: median and skew

In [34]:
dtclassify(vars2nonnan,2,4,'median','skew')

% Class  2  objects in Training: 0.7368281000532197 0.44035491484163325
% Class  4  objects in Testing: 0.7262569832402235 0.44587865785999814
############# based on standard predict ################
Accuracy on training data: 0.92
Accuracy on test data:     0.92
[[ 575  111]
 [  83 1737]]
########################################################
digraph Tree {
node [shape=box] ;
0 [label="X[1] <= 0.668\ngini = 0.388\nsamples = 100.0%\nvalue = [0.263, 0.737]"] ;
1 [label="X[0] <= 18.235\ngini = 0.309\nsamples = 27.9%\nvalue = [0.809, 0.191]"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label="X[1] <= 0.247\ngini = 0.4\nsamples = 17.6%\nvalue = [0.724, 0.276]"] ;
1 -> 2 ;
3 [label="gini = 0.327\nsamples = 14.5%\nvalue = [0.794, 0.206]"] ;
2 -> 3 ;
4 [label="gini = 0.477\nsamples = 3.1%\nvalue = [0.393, 0.607]"] ;
2 -> 4 ;
5 [label="X[1] <= 0.598\ngini = 0.084\nsamples = 10.3%\nvalue = [0.956, 0.044]"] ;
1 -> 5 ;
6 [label="gini = 0.038\nsamples = 9.5%\nvalue = [0.98

[DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort=False,
                        random_state=None, splitter='best'), 'median', 'skew']

## 92% accuracy on train and test classes (see the confusion matrix)! Not bad, eh?

# Now that you know how to add features, subset data and all that fun stuff, do some exercises on your own. Remember, getting a dataset ready is the hardest work! Also remember that our code here is not optimal. If running on bigger sets, generate features more efficiently. 

# (a) add some more features e.g. stddev (look up Richards et al., or Faraway et al. for some additional features)
# (b) run the random forest part on the light curve features